In [1]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

In [2]:
jars = [
    "/Users/ducth/docker/spark-kafka/java/spark-sql-kafka-0-10_2.12-3.5.1.jar",
    "/Users/ducth/docker/spark-kafka/java/kafka-clients-3.7.0.jar,",
    "/Users/ducth/docker/spark-kafka/java/commons-pool2-2.12.0.jar",
    "/Users/ducth/docker/spark-kafka/java/spark-token-provider-kafka-0-10_2.12-3.5.1.jar",
]

conf = SparkConf().set("spark.jars", ",".join(jars))
spark = SparkSession.builder.config(conf=conf).getOrCreate()  # type: ignore

24/03/31 19:58:48 WARN Utils: Your hostname, ThuyMacAir.local resolves to a loopback address: 127.0.0.1; using 192.168.0.11 instead (on interface en0)
24/03/31 19:58:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/31 19:58:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from datetime import datetime, date
from pyspark.sql import Row

df = spark.createDataFrame(
    [
        Row(a=1, b=2.0, c="string1", d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
        Row(a=2, b=3.0, c="string2", d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
        Row(a=4, b=5.0, c="string3", d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0)),
    ]
)
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [9]:
import pyspark.sql.functions as f

df.select(
    [df["a"].cast("string").alias("key"), f.to_json(f.struct("*")).alias("value")]
).show()

+---+--------------------+
|key|               value|
+---+--------------------+
|  1|{"a":1,"b":2.0,"c...|
|  2|{"a":2,"b":3.0,"c...|
|  4|{"a":4,"b":5.0,"c...|
+---+--------------------+



In [4]:
import pyspark.sql.functions as f

(
    df.select(
        [df["a"].cast("string").alias("key"), f.to_json(f.struct("*")).alias("value")]
    )
    .write.format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("topic", "spark-topic")
    .save()
)

24/03/31 19:59:03 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [16]:
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

df = (
    spark.read.format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "spark-topic")
    .option("startingTimestamp", "1711889935964")
    .load()
)


parsed_value_df = df.select(
    f.from_json(
        df["value"].cast("string"),
        StructType(
            [
                StructField("a", IntegerType()),
                StructField("b", StringType()),
                StructField("c", StringType()),
                StructField("d", StringType()),
                StructField("e", StringType()),
            ]
        ),
    ).alias("value")
)

parsed_value_df.select(parsed_value_df["value.a"]).show()

+---+
|  a|
+---+
|  2|
|  4|
+---+



In [6]:
spark.stop()